In [1]:
import pandas as pd
import numpy as np
from random import random
from CTL2.causal_tree_learn import CausalTree
from DGP.dgp4 import dgp4
from MC.mc2 import MC_MSE_2
from MC.mc2_maxd import MC_MSE_2max
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt
import pickle as pkl
#from notify_run import Notify 
#notify = Notify()
import datetime

In [2]:
def MC_ate2(n, var_e, reps, nomin_test):    
    #true ATE
    ATE1 = np.ones(reps)*1.5
    ATE2 = np.ones(reps)*-1.5
    #predicted ate
    ate1 = []
    ate2 = []
    #TOTAL MSE, Bias, Var
    MSE = []
    bias = []
    var = []
    n2 = int(n/2)
    leafs = []
    for i in range(reps):
        x_train, x_test, y_train, y_test, treat_train, treat_test = dgp4(n,var_e)     
        #true individual treatment effect
        #tau = np.where(((x1 >= 0) & (x2 >= 0)),2, np.where(((x1 >= 0) & (x2 < 0)),1, np.where(((x1 < 0) & (x2 >= 0)),-1,-2)))
        tau = np.where((x_test[:,0] >= 0),1.5, -1.5)
        ctl_predict = MC_MSE_2max(x_train, x_test, y_train, y_test, treat_train, treat_test, nomin_test)
        #predicted ATE
        ate1 = np.append(ate1, np.mean(ctl_predict[np.where(x_test[:,0] >= 0)]))
        ate2 = np.append(ate2, np.mean(ctl_predict[np.where(x_test[:,0] < 0)]))
        
        #leafs = np.append(leafs, len(np.unique(ctl_predict)))
        #nn = int(len(ctl_predict))
        MSE = np.append(MSE, mse(ctl_predict,tau))
        bias = np.append(bias, np.sum(np.subtract(tau,ctl_predict))/n2)
        var = np.append(var, np.var(np.subtract(tau,ctl_predict)))

    #ATE MSE's
    mean_ate = np.array([np.mean(ate1), np.mean(ate2)])    
    mse_ate = np.array([mse(ATE1, ate1), mse(ATE2, ate2)])
    bias_ate = np.array([np.sum(np.subtract(ATE1,ate1))/reps, np.sum(np.subtract(ATE2,ate2))/reps])
    var_ate = np.array([np.var(ate1), np.var(ate2)])
    
    return mean_ate, mse_ate, bias_ate, var_ate, np.mean(MSE), np.mean(bias), np.mean(var)

In [4]:
print("start time: ", str(datetime.datetime.now())[11:16])
test_sizes = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

mean_ate_ = []
mse_ate_ = []
bias_ate_ = []
var_ate_ = []
MSE_ = []
bias_ = []
var_ = []

var_e = 0.01
n = 1000
reps = 5

for size in test_sizes:
    mean_ate, mse_ate, bias_ate, var_ate, MSE, bias, var = MC_ate2(n, var_e, reps, size)
    mean_ate_ = np.append(mean_ate_,mean_ate)
    mse_ate_ = np.append(mse_ate_,mse_ate)
    bias_ate_ = np.append(bias_ate_, bias_ate)
    var_ate_ = np.append(var_ate_, var_ate)
    MSE_ = np.append(MSE_, MSE)
    bias_ = np.append(bias_, bias)
    var_ = np.append(var_, var)
    
#notify.send('Done running the simulations')
print("end time: ", str(datetime.datetime.now())[11:16])

start time:  02:39
end time:  02:40


In [20]:
mean_ate_ = mean_ate_.reshape(len(test_sizes),2).transpose()
mse_ate_ = mse_ate_.reshape(len(test_sizes),2).transpose()
bias_ate_ = bias_ate_.reshape(len(test_sizes),2).transpose()
var_ate_ = var_ate_.reshape(len(test_sizes),2).transpose()

In [ ]:
mean_ate_ = mean_ate_.reshape(len(test_sizes),4).transpose()
mse_ate_ = mse_ate_.reshape(len(test_sizes),4).transpose()
bias_ate_ = bias_ate_.reshape(len(test_sizes),4).transpose()
var_ate_ = var_ate_.reshape(len(test_sizes),4).transpose()

In [22]:
mean_ate_ = mean_ate_.reshape(len(test_sizes),4).transpose()
mse_ate_ = mse_ate_.reshape(len(test_sizes),4).transpose()
bias_ate_ = bias_ate_.reshape(len(test_sizes),4).transpose()
var_ate_ = var_ate_.reshape(len(test_sizes),4).transpose()
df = pd.DataFrame([test_sizes, mean_ate_[0], mean_ate_[1],mean_ate_[2], mean_ate_[3], mse_ate_[0], mse_ate_[1],mse_ate_[2], mse_ate_[3], bias_ate_[0], bias_ate_[1], bias_ate_[2], bias_ate_[3], var_ate_[0], var_ate_[1],var_ate_[2], var_ate_[3], MSE_, bias_, var_]).transpose().set_index(0)
df.columns=['Mean1', 'Mean2', 'Mean3', 'Mean4', 'MSE1', 'MSE2','MSE3', 'MSE4', 'BIAS1', 'BIAS2','BIAS3', 'BIAS4', 'VAR1', 'VAR2','VAR3', 'VAR4', 'MSE_TOTAL', 'MSE_T_BIAS', 'MSE_T_VAR']

In [23]:
df

,Mean1,Mean2,MSE1,MSE2,BIAS1,BIAS2,VAR1,VAR2,MSE_TOTAL,MSE_T_BIAS,MSE_T_VAR
0,,,,,,,,,,,
0.2,1.691971,-1.481534,0.101903,0.038320,-0.191971,-0.018466,0.065050,0.037979,0.239861,-0.107464,0.222237
0.3,1.410150,-1.452264,0.024322,0.023429,0.089850,-0.047736,0.016249,0.021151,0.287408,0.022775,0.281377
0.4,1.169408,-1.195357,0.143118,0.199800,0.330592,-0.304643,0.033827,0.106993,0.337511,0.016832,0.290455
0.5,1.507453,-1.388777,0.082896,0.159450,-0.007453,-0.111223,0.082840,0.147080,0.268964,-0.055953,0.181218
0.6,1.239287,-1.173621,0.186332,0.245199,0.260713,-0.326379,0.118361,0.138676,0.412817,-0.029030,0.389061
0.7,1.385854,-1.350041,0.034143,0.123779,0.114146,-0.149959,0.021114,0.101292,0.381972,-0.020549,0.361521
0.8,0.839287,-1.011267,0.739157,0.724798,0.660713,-0.488733,0.302615,0.485938,0.963052,0.072313,0.949001
0.9,0.875679,-0.782582,0.689882,0.831784,0.624321,-0.717418,0.300106,0.317095,1.070842,-0.053193,1.042217


In [31]:
results = [['var = 0.1, reps = 500, n = 1000, with int.'],[mean_ate_], [mse_ate_], [bias_ate_], [var_ate_], [MSE_], [bias_], [var_]]
with open('te5_r1_.pkl','wb') as f:
    pkl.dump(results, f)

In [ ]:
df = pd.DataFrame([test_sizes, MSE1, MSE2, BIAS1, BIAS2, VAR1, VAR2, MSE_TOTAL, MSEB, MSEV]).transpose().set_index(0)
df.columns=['MSE1', 'MSE2', 'BIAS1', 'BIAS2', 'VAR1', 'VAR2', 'MSE_TOTAL', 'MSE_T_BIAS', 'MSE_T_VAR']

In [2]:
import pickle as pkl
with open('te5_r1.pkl', 'rb') as f:
    df1 = pkl.load(f)